In [6]:
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
from scipy.stats import multivariate_normal
import random

In [35]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = np.load('./fineweb_dataset_embeded.npy')
print(dataset.shape)

(998, 1536)


## Incremental EM

In [53]:
# Reducted
# n = dataset.shape[0]

# def compute_posterior(x, mu, cov, weights):
#     n_centers = len(mu)
#     posterior = np.zeros(n_centers)
#     for i in range(n_centers):
#         posterior[i] = weights[i] * multivariate_normal.pdf(x, mu[i], cov[i])
#     return np.sum(posterior)

# # Hyperparameters
# T = 5
# n_centers = 3

# # Initialization
# gaussian_dist = GaussianMixture(n_components = n_centers)
# gaussian_dist.fit(dataset)

# means = gaussian_dist.means_
# covariances = gaussian_dist.covariances_
# weights = gaussian_dist.weights_

# si = np.zeros(n)
# global_mu = np.sum(si, axis=0)

# # Iterations
# for t in range(T):
#     order = np.random.permutation(n)
#     for i in order:
#         s0_i = compute_posterior(dataset[i], means, covariances, weights)
#         global_mu += s0_i - si[i]
#         si[i] = s0_i

In [ ]:
# E-step: Compute responsibilities (posterior probabilities) for each data point
def e_step(X, mu, covariances, weights, n_components):
    n_samples = X.shape[0]
    responsibilities = np.zeros((n_samples, n_components))

    for i in range(n_samples):
        total_prob = 0.0
        # Calculate the probability for each component and normalize to get responsibilities
        for k in range(n_components):
            pdf_val = multivariate_normal.pdf(X[i], mean=mu[k], cov=covariances[k])
            responsibilities[i, k] = weights[k] * pdf_val
            total_prob += responsibilities[i, k]

        # Normalize to get the posterior probabilities (responsibilities)
        responsibilities[i, :] /= total_prob
    
    return responsibilities

# M-step: Update the parameters of the GMM based on the responsibilities
def m_step(X, responsibilities, mu, covariances, weights, n_components):
    n_samples, n_features = X.shape

    # Update the parameters incrementally
    for k in range(n_components):
        # Compute the total responsibility for component k
        total_responsibility = np.sum(responsibilities[:, k])

        # Update the weight (prior) for component k
        weights[k] = total_responsibility / n_samples

        # Update the mean (mu) for component k (weighted average of the data points)
        mu[k] = np.sum(responsibilities[:, k][:, np.newaxis] * X, axis=0) / total_responsibility

        # Update the covariance matrix (weighted covariance)
        cov_matrix = np.zeros((n_features, n_features))
        for i in range(n_samples):
            diff = X[i] - mu[k]
            cov_matrix += responsibilities[i, k] * np.outer(diff, diff)

        covariances[k] = cov_matrix / total_responsibility

    return mu, covariances, weights

# Incremental EM Algorithm
def incremental_em(X, n_components, max_iter=100, tol=1e-6):
    n_samples, n_features = X.shape
    
    # Initialize the GMM parameters (means, covariances, and weights)
    np.random.seed(42)
    mu = np.random.randn(n_components, n_features)
    covariances = np.array([np.eye(n_features)] * n_components)  # Identity covariance for each component
    weights = np.ones(n_components) / n_components  # Equal weight for each component initially
    
    # Initialize responsibilities (posterior probabilities) for each data point
    responsibilities = np.zeros((n_samples, n_components))

    # Iterative process
    prev_log_likelihood = -np.inf
    for t in range(max_iter):
        print(f"Iteration {t + 1}/{max_iter}")

        # E-step: Compute responsibilities for each data point
        responsibilities = e_step(X, mu, covariances, weights, n_components)

        # M-step: Update the parameters of the GMM
        mu, covariances, weights = m_step(X, responsibilities, mu, covariances, weights, n_components)

        # Compute the log-likelihood for convergence check
        log_likelihood = np.sum(np.log(np.sum(responsibilities, axis=1)))
        print(f"Log-likelihood at iteration {t + 1}: {log_likelihood}")

        # Check for convergence (if log-likelihood changes by less than tol, stop)
        if np.abs(log_likelihood - prev_log_likelihood) < tol:
            print("Converged!")
            break

        prev_log_likelihood = log_likelihood

    return mu, covariances, weights

# Example usage
if __name__ == "__main__":
    # Generate synthetic data (for demonstration purposes)
    from sklearn.datasets import make_blobs
    
    X = dataset
    
    # Run Incremental EM
    n_components = 3
    mu, covariances, weights = incremental_em(X, n_components=n_components, max_iter=50)

    print("Final means (mu):")
    print(mu)
    print("Final covariances (Sigma):")
    print(covariances)
    print("Final weights (pi):")
    print(weights)


In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
from typing import List, Tuple

class IncrementalGMM:
    def __init__(self, n_components: int, dim: int, learning_rate: float = 0.1):
        """
        Initialize Incremental Gaussian Mixture Model
        
        Args:
            n_components: Number of Gaussian components
            dim: Dimensionality of the data
            learning_rate: Learning rate for parameter updates
        """
        self.n_components = n_components
        self.dim = dim
        self.learning_rate = learning_rate
        
        # Initialize parameters
        self.weights = np.ones(n_components) / n_components
        self.means = np.random.randn(n_components, dim)
        self.covs = np.array([np.eye(dim) for _ in range(n_components)])
        
    def _compute_responsibilities(self, x: np.ndarray) -> np.ndarray:
        """
        Compute responsibilities (E-step) for a single data point
        
        Args:
            x: Single data point of shape (dim,)
            
        Returns:
            responsibilities: Array of shape (n_components,)
        """
        densities = np.array([
            multivariate_normal.pdf(x, mean=mean, cov=cov)
            for mean, cov in zip(self.means, self.covs)
        ])
        
        weighted_densities = self.weights * densities
        sum_densities = weighted_densities.sum()
        
        if sum_densities == 0:
            return np.ones(self.n_components) / self.n_components
            
        return weighted_densities / sum_densities
    
    def _update_parameters(self, x: np.ndarray, responsibilities: np.ndarray):
        """
        Update parameters (M-step) using a single data point
        
        Args:
            x: Single data point of shape (dim,)
            responsibilities: Array of shape (n_components,)
        """
        for k in range(self.n_components):
            resp_k = responsibilities[k]
            
            # Update weight
            self.weights[k] = (1 - self.learning_rate) * self.weights[k] + \
                            self.learning_rate * resp_k
            
            # Update mean
            diff_mean = resp_k * (x - self.means[k])
            self.means[k] += self.learning_rate * diff_mean
            
            # Update covariance
            diff_outer = np.outer(x - self.means[k], x - self.means[k])
            diff_cov = resp_k * (diff_outer - self.covs[k])
            self.covs[k] += self.learning_rate * diff_cov
            
        # Normalize weights
        self.weights /= self.weights.sum()
        
        # Ensure numerical stability of covariances
        for k in range(self.n_components):
            self.covs[k] = (self.covs[k] + self.covs[k].T) / 2  # Ensure symmetry
            min_eig = np.min(np.real(np.linalg.eigvals(self.covs[k])))
            if min_eig < 1e-6:
                self.covs[k] += (1e-6 - min_eig) * np.eye(self.dim)
    
    def partial_fit(self, x: np.ndarray):
        """
        Update the model with a single data point
        
        Args:
            x: Single data point of shape (dim,)
        """
        if x.shape[0] != self.dim:
            raise ValueError(f"Expected data point of dimension {self.dim}, got {x.shape[0]}")
        
        # E-step
        responsibilities = self._compute_responsibilities(x)
        
        # M-step
        self._update_parameters(x, responsibilities)
    
    def fit(self, X: np.ndarray, n_epochs: int = 1):
        """
        Fit the model to the data using multiple passes
        
        Args:
            X: Data matrix of shape (n_samples, dim)
            n_epochs: Number of passes through the data
        """
        if X.shape[1] != self.dim:
            raise ValueError(f"Expected data of dimension {self.dim}, got {X.shape[1]}")
        
        for epoch in range(n_epochs):
            # Shuffle data
            np.random.shuffle(X)
            
            # Process each point
            for x in X:
                self.partial_fit(x)
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Compute responsibilities for multiple data points
        
        Args:
            X: Data matrix of shape (n_samples, dim)
            
        Returns:
            responsibilities: Array of shape (n_samples, n_components)
        """
        return np.array([self._compute_responsibilities(x) for x in X])
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict cluster assignments for multiple data points
        
        Args:
            X: Data matrix of shape (n_samples, dim)
            
        Returns:
            assignments: Array of shape (n_samples,)
        """
        return self.predict_proba(X).argmax(axis=1)

In [ ]:
# Create synthetic data
np.random.seed(42)
n_samples = 1000
X = np.concatenate([
    np.random.normal(0, 1, (n_samples // 2, 2)),
    np.random.normal(4, 1.5, (n_samples // 2, 2))
])

# Initialize and fit the model
gmm = IncrementalGMM(n_components=2, dim=2)
gmm.fit(X, n_epochs=3)

# Make predictions
predictions = gmm.predict(X)